In [1]:
# 载入原始用户移动端行为数据
import pandas as pd
base_path = '/home/liyouru/.ipython/profile_myserver/Dataset/JDD/'
path_t_user = base_path + 'user.csv'
path_t_order = base_path + 't_order.csv'
path_t_loan_sum = base_path + 't_loan_sum.csv'
path_t_loan = base_path + 't_loan.csv'
path_t_click = base_path + 't_click.csv'
path_submit_sample = base_path + 'Loan_Forecasting.csv'

# 用户基本信息
user_df = pd.read_csv(path_t_user, header=0, sep=',')
# 购买记录
order_df = pd.read_csv(path_t_order, header=0, sep=',')
# 点击记录
click_df = pd.read_csv(path_t_click,header=0, sep=',')
# 借贷历史
loan_df = pd.read_csv(path_t_loan,header=0, sep=',')
# 借贷月度统计
loan_sum_df = pd.read_csv(path_t_loan_sum,header=0, sep=',')
# 提交用户uid
uid = []
with open(path_submit_sample, 'rb') as f:
    f.readline()
    for line in f:
        line = line.strip().split(',')
        uid.append(int(line[0]))
        
def show_data_stru():
    print (user_df.shape)
    print user_df.head()
    print '-------------------------------------------------------------'
    print (order_df.shape)
    print order_df.head()
    print '-------------------------------------------------------------'
    print (click_df.shape)
    print click_df.head()
    print '-------------------------------------------------------------'
    print (loan_df.shape)
    print loan_df.head()
    print '-------------------------------------------------------------'
    print (loan_sum_df.shape)
    print loan_sum_df.head()
    
#show_data_stru()
from __future__ import division
import numpy as np
import datetime
from tqdm import tqdm
import time
import math

base_feature = []

# 年龄——0
base_feature.append(list(user_df['age']))
# 性别——1
base_feature.append(list(user_df['sex']))
# 限额——2
base_feature.append(list(user_df['limit']))

def detla_date(date,T_month,T_day):
    d1 = date
    d2 = datetime.datetime(2016, T_month, T_day)
    return (d1 - d2).days

active_duration = []
for each in list(user_df['active_date']):
    if each == '0' or 'nan':
        each = '2016/5/1'
    active_duration.append(abs(detla_date(datetime.datetime.strptime(each, "%Y/%m/%d"),11,30)))

# 激活时长——3
base_feature.append(active_duration)

print len(base_feature)

loan_time = list(loan_df['loan_time'])
loan_amount = list(loan_df['loan_amount'])
loan_plannum = list(loan_df['plannum'])
loan_uid_temp = list(loan_df['uid'])
loan_uid = []
for i in range(len(loan_uid_temp)):
    loan_uid.append(int(loan_uid_temp[i]))

month_index_loan = ['08','09','10','11']

def get_loan_feature(loan_uid,loan_time,loan_amount,month_index_loan):
    loan_monthly_feature = []
    uid_loan_map = {}
    
    # 初始化一键多值map
    for i in range(len(uid)):
        for j in range(12):
            uid_loan_map.setdefault(uid[i],[]).append(0)
    
    # 计算每个uid月度点击总量
    for i in tqdm(range(len(loan_uid))):
        if loan_time[i][5:7] == month_index_loan[0]:
            # 钱数
            uid_loan_map[loan_uid[i]][0] += loan_amount[i]
            # 次数
            uid_loan_map[loan_uid[i]][1] += 1
            # 期数
            uid_loan_map[loan_uid[i]][2] += loan_plannum[i]
        elif loan_time[i][5:7] == month_index_loan[1]:
            # 钱数
            uid_loan_map[loan_uid[i]][3] += loan_amount[i]
            # 次数
            uid_loan_map[loan_uid[i]][4] += 1
            # 期数
            uid_loan_map[loan_uid[i]][5] += loan_plannum[i]        
        elif loan_time[i][5:7] == month_index_loan[2]:
            # 钱数
            uid_loan_map[loan_uid[i]][6] += loan_amount[i]
            # 次数
            uid_loan_map[loan_uid[i]][7] += 1
            # 期数
            uid_loan_map[loan_uid[i]][8] += loan_plannum[i]
        elif loan_time[i][5:7] == month_index_loan[3]:
            # 钱数
            uid_loan_map[loan_uid[i]][9] += loan_amount[i]
            # 次数
            uid_loan_map[loan_uid[i]][10] += 1
            # 期数
            uid_loan_map[loan_uid[i]][11] += loan_plannum[i]
    
    # 保存点击量特征向量
    c_feature_1 = []
    c_feature_2 = []
    c_feature_3 = []
    c_feature_4 = []
    c_feature_5 = []
    c_feature_6 = []
    c_feature_7 = []
    c_feature_8 = []
    c_feature_9 = []
    c_feature_10 = []
    c_feature_11 = []
    c_feature_12 = []
    
    for i in range(len(uid)):
        c_feature_1.append(uid_loan_map[uid[i]][0])
        c_feature_2.append(uid_loan_map[uid[i]][1])
        if uid_loan_map[uid[i]][1] != 0:
            c_feature_3.append(int(uid_loan_map[uid[i]][2]/uid_loan_map[uid[i]][1]))
        else:
            c_feature_3.append(0)
        c_feature_4.append(uid_loan_map[uid[i]][3])
        c_feature_5.append(uid_loan_map[uid[i]][4])
        if uid_loan_map[uid[i]][4] != 0:
            c_feature_6.append(int(uid_loan_map[uid[i]][5]/uid_loan_map[uid[i]][4]))
        else:
            c_feature_6.append(0)
        c_feature_7.append(uid_loan_map[uid[i]][6])
        c_feature_8.append(uid_loan_map[uid[i]][7])
        if uid_loan_map[uid[i]][7] != 0:
            c_feature_9.append(int(uid_loan_map[uid[i]][8]/uid_loan_map[uid[i]][7]))
        else:
            c_feature_9.append(0)
        c_feature_10.append(uid_loan_map[uid[i]][9])
        c_feature_11.append(uid_loan_map[uid[i]][10])
        if uid_loan_map[uid[i]][10] != 0:
            c_feature_12.append(int(uid_loan_map[uid[i]][11]/uid_loan_map[uid[i]][10]))
        else:
            c_feature_12.append(0)

    # 并入时序特征
    loan_monthly_feature.append(c_feature_1)
    loan_monthly_feature.append(c_feature_2)
    loan_monthly_feature.append(c_feature_3)
    loan_monthly_feature.append(c_feature_4)
    loan_monthly_feature.append(c_feature_5)
    loan_monthly_feature.append(c_feature_6)
    loan_monthly_feature.append(c_feature_7)
    loan_monthly_feature.append(c_feature_8)
    loan_monthly_feature.append(c_feature_9)
    loan_monthly_feature.append(c_feature_10)
    loan_monthly_feature.append(c_feature_11)
    loan_monthly_feature.append(c_feature_12)
    return loan_monthly_feature

loan_feature = get_loan_feature(loan_uid,loan_time,loan_amount,month_index_loan)

loan_8_10 = []
loan_9_11 = []

loan_8_10.extend(loan_feature[:9])
loan_9_11.extend(loan_feature[3:12])
print len(loan_8_10)
print len(loan_9_11)

# df_loan_8_10 = pd.DataFrame(np.transpose(np.asarray(loan_8_10)))
# df_loan_8_10.to_csv(base_path + 'df_loan_8_10.csv')

# df_loan_9_11 = pd.DataFrame(np.transpose(np.asarray(loan_9_11)))
# df_loan_9_11.to_csv(base_path + 'df_loan_9_11.csv')

order_time = list(order_df['buy_time'])
order_qty = list(order_df['qty'])
order_price = list(order_df['price'])
order_discount = list(order_df['discount'])

order_uid_temp = list(order_df['uid'])
order_uid = []
for i in range(len(order_uid_temp)):
    order_uid.append(int(order_uid_temp[i]))

month_index_order = ['08','09','10','11']

def get_order_feature(click_uid,click_time,order_price,order_qty,month_index_order):
    order_monthly_feature = []
    uid_order_map = {}
    
    # 初始化一键多值map
    for i in range(len(uid)):
        for j in range(4):
            uid_order_map.setdefault(uid[i],[]).append(0)
    
    # 计算每个uid月度点击总量
    for i in tqdm(range(len(order_uid))):
        if order_time[i][5:7] == month_index_order[0]:
            uid_order_map[order_uid[i]][0] += order_price[i] * order_qty[i] - order_discount[i]
        elif order_time[i][5:7] == month_index_order[1]:
            uid_order_map[order_uid[i]][1] += order_price[i] * order_qty[i] - order_discount[i]
        elif order_time[i][5:7] == month_index_order[2]:
            uid_order_map[order_uid[i]][2] += order_price[i] * order_qty[i] - order_discount[i]
        elif order_time[i][5:7] == month_index_order[3]:
            uid_order_map[order_uid[i]][3] += order_price[i] * order_qty[i] - order_discount[i]
    
    # 保存点击量特征向量
    c_feature_1 = []
    c_feature_2 = []
    c_feature_3 = []
    c_feature_4 = []
    
    for i in range(len(uid)):
        c_feature_1.append(uid_order_map[uid[i]][0])
        c_feature_2.append(uid_order_map[uid[i]][1])
        c_feature_3.append(uid_order_map[uid[i]][2])
        c_feature_4.append(uid_order_map[uid[i]][3])

    # 并入时序特征
    order_monthly_feature.append(c_feature_1)
    order_monthly_feature.append(c_feature_2)
    order_monthly_feature.append(c_feature_3)
    order_monthly_feature.append(c_feature_4)
    return order_monthly_feature

order_feature = get_order_feature(order_uid,order_time,order_price,order_qty,month_index_order)

order_8_10 = []
order_9_11 = []

order_8_10.extend(order_feature[:3])
order_9_11.extend(order_feature[1:4])

# 缺失值补充
def get_nan2zero(souce_list):
    for each in souce_list:
        for i in range(len(each)):
            for j in range(len(each[0])):
                if math.isnan(each[i][j]):
                    each[i][j] = 0

souce_list = [order_8_10,order_9_11]
get_nan2zero(souce_list)

print len(order_8_10)
print len(order_8_10)

# df_order_8_10 = pd.DataFrame(np.transpose(np.asarray(order_8_10)))
# df_order_8_10.to_csv(base_path + 'df_order_8_10.csv')

# df_order_9_11 = pd.DataFrame(np.transpose(np.asarray(order_9_11)))
# df_order_9_11.to_csv(base_path + 'df_order_9_11.csv')

order_cata_id_temp = list(order_df['cate_id'])
order_cata_id = []
for each in order_cata_id_temp:
    order_cata_id.append(int(each))

# print len(order_cata_id)
# res={}
# for i in order_cata_id:
#     res[i] = res.get(i,0)+1
# print res

month_index_order = ['08','09','10','11']

def get_order_cata_id_feature(order_uid,order_time,order_cata_id,order_qty,month_index_order):
    order_cata_id_monthly_feature = []
    uid_order_cata_id_map = {}
    
    # 初始化一键多值map
    for i in range(len(uid)):
        for j in range(176):
            uid_order_cata_id_map.setdefault(uid[i],[]).append(0)
    
    # 计算每个uid月度cata_id的one-hot特征
    for i in tqdm(range(len(order_uid))):
        if order_time[i][5:7] == month_index_order[0]:
            uid_order_cata_id_map[order_uid[i]][order_cata_id[i] - 1] += order_qty[i]
        elif order_time[i][5:7] == month_index_order[1]:
            uid_order_cata_id_map[order_uid[i]][order_cata_id[i] - 1 + 44] += order_qty[i]
        elif order_time[i][5:7] == month_index_order[2]:
            uid_order_cata_id_map[order_uid[i]][order_cata_id[i] - 1 + 88] += order_qty[i]
        elif order_time[i][5:7] == month_index_order[3]:
            uid_order_cata_id_map[order_uid[i]][order_cata_id[i] - 1 + 132] += order_qty[i]
    
    for i in range(176):
        temp = []
        for j in range(len(uid)):
            temp.append(uid_order_cata_id_map[uid[j]][i])
        order_cata_id_monthly_feature.append(temp)
    
    return order_cata_id_monthly_feature

order_cata_id_feature = get_order_cata_id_feature(order_uid,order_time,order_cata_id,order_qty,month_index_order)

order_cata_id_8_10 = []
order_cata_id_9_11 = []

order_cata_id_8_10.extend(order_cata_id_feature[:132])
order_cata_id_9_11.extend(order_cata_id_feature[44:])

print len(order_cata_id_8_10)
print len(order_cata_id_9_11)

# df_order_cata_id_8_10 = pd.DataFrame(np.transpose(np.asarray(order_cata_id_8_10)))
# df_order_cata_id_8_10.to_csv(base_path + 'df_order_cata_id_8_10.csv')

# df_order_cata_id_9_11 = pd.DataFrame(np.transpose(np.asarray(order_cata_id_9_11)))
# df_order_cata_id_9_11.to_csv(base_path + 'df_order_cata_id_9_11.csv')

click_time = list(click_df['click_time'])

click_uid_temp = list(click_df['uid'])
click_uid = []
for i in range(len(click_uid_temp)):
    click_uid.append(int(click_uid_temp[i]))

month_index_order = ['08','09','10','11']

def get_click_feature(click_uid,click_time,month_index_order):
    click_monthly_feature = []
    uid_click_map = {}
    
    # 初始化一键多值map
    for i in range(len(uid)):
        for j in range(4):
            uid_click_map.setdefault(uid[i],[]).append(0)
    
    # 计算每个uid月度点击总量
    for i in tqdm(range(len(click_uid))):
        if click_time[i][5:7] == month_index_order[0]:
            uid_click_map[click_uid[i]][0] += 1
        elif click_time[i][5:7] == month_index_order[1]:
            uid_click_map[click_uid[i]][1] += 1
        elif click_time[i][5:7] == month_index_order[2]:
            uid_click_map[click_uid[i]][2] += 1
        elif click_time[i][5:7] == month_index_order[3]:
            uid_click_map[click_uid[i]][3] += 1
    
    # 保存点击量特征向量
    c_feature_1 = []
    c_feature_2 = []
    c_feature_3 = []
    c_feature_4 = []
    
    for i in range(len(uid)):
        c_feature_1.append(uid_click_map[uid[i]][0])
        c_feature_2.append(uid_click_map[uid[i]][1])
        c_feature_3.append(uid_click_map[uid[i]][2])
        c_feature_4.append(uid_click_map[uid[i]][3])

    # 并入时序特征
    click_monthly_feature.append(c_feature_1)
    click_monthly_feature.append(c_feature_2)
    click_monthly_feature.append(c_feature_3)
    click_monthly_feature.append(c_feature_4)
    return click_monthly_feature

click_feature = get_click_feature(click_uid,click_time,month_index_order)

click_8_10 = []
click_9_11 = []

click_8_10.extend(click_feature[:3])
click_9_11.extend(click_feature[1:4])
print len(click_8_10)
print len(click_9_11)

# df_click_8_10 = pd.DataFrame(np.transpose(np.asarray(click_8_10)))
# df_click_8_10.to_csv(base_path + 'df_click_8_10.csv')

# df_click_9_11 = pd.DataFrame(np.transpose(np.asarray(click_9_11)))
# df_click_9_11.to_csv(base_path + 'df_click_9_11.csv')

click_pid_temp = list(click_df['pid'])
click_pid = []
for each in click_pid_temp:
    click_pid.append(int(each))

click_param_temp = list(click_df['param'])
click_param = []
for each in click_param_temp:
    click_param.append(int(each))

# 获得所有pid_param pairs
pid_param_map = {}
for i in tqdm(range(len(click_pid))):
    pid_param_map.setdefault(click_pid[i],[]).append(click_param[i])
    
# 除重
pid_param_pair = []
for i in range(10):
    pid_param_pair.append(list(set(pid_param_map[i + 1])))

# 多键一值map: pid + param --> index
pid_param_pair_order = {}
index = 0
for i in range(10):
    for each in pid_param_pair[i]:
        pid_param_pair_order[i + 1,each] = index
        index += 1

# 窗口月份表
month_index_click = ['08','09','10','11']

def get_click_pid_param_cross_feature(click_uid,click_time,click_pid,click_param,month_index_click):
    click_pid_monthly_feature = []
    uid_click_pid_map = {}
    
    # 初始化一键多值map
    for i in range(len(uid)):
        for j in range(692):
            uid_click_pid_map.setdefault(uid[i],[]).append(0)
    
    # 计算每个uid月度click_pid_param的one-hot交叉特征
    for i in tqdm(range(len(click_uid))):
        if click_time[i][5:7] == month_index_click[0]:
            uid_click_pid_map[click_uid[i]][pid_param_pair_order[click_pid[i],click_param[i]]] += 1
        elif click_time[i][5:7] == month_index_click[1]:
            uid_click_pid_map[click_uid[i]][pid_param_pair_order[click_pid[i],click_param[i]] + 173] += 1
        elif click_time[i][5:7] == month_index_click[2]:
            uid_click_pid_map[click_uid[i]][pid_param_pair_order[click_pid[i],click_param[i]] + 346] += 1
        elif click_time[i][5:7] == month_index_click[3]:
            uid_click_pid_map[click_uid[i]][pid_param_pair_order[click_pid[i],click_param[i]] + 519] += 1
    
    for i in range(692):
        temp = []
        for j in range(len(uid)):
            temp.append(uid_click_pid_map[uid[j]][i])
        click_pid_monthly_feature.append(temp)
    
    return click_pid_monthly_feature

click_pid_feature = get_click_pid_param_cross_feature(click_uid,click_time,click_pid,click_param,month_index_click)

click_pid_8_10 = []
click_pid_9_11 = []

click_pid_8_10.extend(click_pid_feature[:519])
click_pid_9_11.extend(click_pid_feature[173:])

print len(click_pid_8_10)
print len(click_pid_9_11)

# df_click_pid_8_10 = pd.DataFrame(np.transpose(np.asarray(click_pid_8_10)))
# df_click_pid_8_10.to_csv(base_path + 'df_click_pid_8_10.csv')

# df_click_pid_9_11 = pd.DataFrame(np.transpose(np.asarray(click_pid_9_11)))
# df_click_pid_9_11.to_csv(base_path + 'df_click_pid_9_11.csv')

loan_sum_map = {}
loan_sum = list(loan_sum_df['loan_sum'])
loan_id = list(loan_sum_df['uid'])
for i in range(len(loan_id)):
    loan_id[i] = int(loan_id[i])

VALUE = []
for i in range(len(uid)):
    VALUE.append(0)

for i in range(len(loan_sum)):
    loan_sum_map[loan_id[i]] = loan_sum[i]

for i in range(len(uid)):
    if uid[i] in loan_id:
        VALUE[i] = loan_sum_map[uid[i]]
        
print len(VALUE)

# 数据准备(特征合并)
feature = []
feature.extend(base_feature)
feature.extend(loan_8_10)
feature.extend(order_8_10)
feature.extend(click_8_10)
feature.extend(order_cata_id_8_10)
feature.extend(click_pid_8_10)

feature_t = []
feature_t.extend(base_feature)
feature_t.extend(loan_9_11)
feature_t.extend(order_9_11)
feature_t.extend(click_9_11)
feature_t.extend(order_cata_id_9_11)
feature_t.extend(click_pid_9_11)


X = np.transpose(np.asarray(feature))
y = np.asarray(VALUE)
X_t = np.transpose(np.asarray(feature_t))

print X.shape
print y.shape
print X_t.shape

# 保存特征数据
# df_X = pd.DataFrame(X)
# df_y = pd.DataFrame(y)
# df_X_t = pd.DataFrame(X_t)

# df_X.to_csv(base_path + 'save_feature_X.csv')
# df_y.to_csv(base_path + 'save_feature_y.csv')
# df_X_t.to_csv(base_path + 'save_feature_X_t.csv')

4


100%|██████████| 202902/202902 [00:00<00:00, 556756.33it/s]


9


100%|██████████| 5400778/5400778 [00:07<00:00, 728776.46it/s]


9
3
3


...

132
132


100%|██████████| 10933016/10933016 [00:07<00:00, 1485641.98it/s]


3
3


100%|██████████| 10933016/10933016 [00:10<00:00, 1057124.67it/s]


519
519
90993
(90993, 670)
(90993,)
(90993, 670)


In [23]:
# 数据准备(特征合并)
feature = []
feature.extend(base_feature)
feature.extend(loan_8_10)
feature.extend(order_8_10)
feature.extend(click_8_10)
feature.extend(order_cata_id_8_10)
feature.extend(click_pid_8_10)

feature_t = []
feature_t.extend(base_feature)
feature_t.extend(loan_9_11)
feature_t.extend(order_9_11)
feature_t.extend(click_9_11)
feature_t.extend(order_cata_id_9_11)
feature_t.extend(click_pid_9_11)


X = np.transpose(np.asarray(feature))
y = np.asarray(VALUE)
X_t = np.transpose(np.asarray(feature_t))

print X.shape
print y.shape
print X_t.shape

(90993, 670)
(90993,)
(90993, 670)


In [27]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import scipy as sp
import time

# 初始化基模型
m_gbrt1 = RandomForestRegressor(n_estimators = 200,max_depth = 4, n_jobs = 8)
m_gbrt2 = RandomForestRegressor(n_estimators = 250,max_depth = 5, n_jobs = 8)
m_rf1 = RandomForestRegressor(n_estimators = 200,max_depth = 5, n_jobs = 8)
m_rf2 = RandomForestRegressor(n_estimators = 300,max_depth = 6, n_jobs = 8)

# 训练集、验证集划分
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state = 0)
print (X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

# Staking 模型
def get_stacking_regressor_feature(split_num, X_train, Y_train, X_test):
    gbrt1 = []
    gbrt2 = []
    rf1 = []
    rf2 = []

    gbrt11 = []
    gbrt22 = []
    rf11 = []
    rf22 = []

    Stacking_Feature = [] 
    kf = KFold(n_splits = split_num)

    # 使用k折方法对基模型的预测结果整合成新的特征矩阵并再次训练7个基模型
    t0 = time.time()
    for train_index, test_index in kf.split(X_train):
        # 训练基模型
        print 'training...'
        m_gbrt1.fit(X_train[train_index],Y_train[train_index])
        print 'training time 1:',time.time() - t0
        m_gbrt2.fit(X_train[train_index],Y_train[train_index])
        print 'training time 2:',time.time() - t0
        m_rf1.fit(X_train[train_index],Y_train[train_index])
        print 'training time 3:',time.time() - t0
        m_rf2.fit(X_train[train_index],Y_train[train_index])
        print 'training time 4:',time.time() - t0

        # 得到基模型上的预测结果
        gbrt1.extend(m_gbrt1.predict(X_train[test_index])) 
        gbrt2.extend(m_gbrt2.predict(X_train[test_index]))
        rf1.extend(m_rf1.predict(X_train[test_index]))
        rf2.extend(m_rf2.predict(X_train[test_index]))

        # 在测试集上得到预测
        gbrt11.append(m_gbrt1.predict(X_test)) 
        gbrt22.append(m_gbrt2.predict(X_test))
        rf11.append(m_rf1.predict(X_test))
        rf22.append(m_rf2.predict(X_test))

    index = [gbrt1, gbrt2, rf1, rf2]
    for i in range(split_num):
        Stacking_Feature.append(index[i])
    
    # 训练集stacking特征
    result = []
    result.append(np.transpose(np.asarray(Stacking_Feature)))
    
    index2 = [gbrt11, gbrt22, rf11, rf22]
    Test_Feature = []
    for each in index2:
        temp = []
        for i in range(18199):
            temp.append((each[0][i] + each[1][i] + each[2][i] + each[3][i]) / 4)
        Test_Feature.append(temp)
    
    # 测试集stacking特征
    result.append(np.transpose(np.asarray(Test_Feature)))
    return result

def RMSE(Y_test, Y_Vali):  
    return sp.sqrt(sp.mean((Y_test - Y_Vali) ** 2))  

((72794, 670), (18199, 670), (72794,), (18199,))


In [28]:
# 再次训练基模型
stacking_result = []
stacking_result = get_stacking_regressor_feature(4, X_train, Y_train, X_test)
print stacking_result[0].shape,stacking_result[1].shape
m_gbrt1.fit(stacking_result[0],Y_train)
Y_Vali = m_gbrt1.predict(stacking_result[1])

print RMSE(Y_test, Y_Vali)
# # # 在测试集上输出预测结果
# # GBRT.fit(X,y)
# # y_t = GBRT.predict(X_t)

# value_predict = []
# for i in range(len(uid)):
#     value_predict.append(max(0,float(y_t[i])))

# with open(base_path + 'submit_1211.csv', 'wb') as f:
#     for i in range(len(uid)):
#         f.write('{},{}\n'.format(uid[i], value_predict[i]))
        
# print 'done!'

training...
training time 1: 55.1191208363
training time 2: 146.490036964
training time 3: 211.461251974
training time 4: 323.02481389
training...
training time 1: 383.46083498
training time 2: 468.656931877
training time 3: 536.771714926
training time 4: 645.664548874
training...
training time 1: 700.214148045
training time 2: 779.337954998
training time 3: 842.926918983
training time 4: 1062.70441198
training...
training time 1: 1178.02826595
training time 2: 1348.64397693
training time 3: 1481.41004181
training time 4: 1707.66443682
(72794, 4) (18199, 4)
1.80678745831


In [35]:
GBRT = GradientBoostingRegressor(n_estimators=200, max_depth=4)
best = RandomForestRegressor(n_estimators = 500,max_depth = 5, n_jobs = 8)
GBRT.fit(stacking_result[0],Y_train)
y_t = GBRT.predict(stacking_result[1])
print RMSE(Y_test, y_t)

1.81557535053
